In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
# from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime

In [7]:
STOCK_CODE = 'KS200'
stock = fdr.DataReader(STOCK_CODE)
stock.iloc[:-3000]

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2000-12-26,63.35,64.11,64.19,63.04,141880000.0,0.0083
2001-01-02,65.41,63.33,65.47,62.97,130590000.0,0.0325
2001-01-03,65.38,64.28,65.79,63.99,182700000.0,-0.0005
2001-01-04,70.14,69.19,71.37,69.19,265500000.0,0.0728
2001-01-05,73.15,70.32,73.23,69.74,352410000.0,0.0429
...,...,...,...,...,...,...
2009-06-11,181.43,180.49,183.84,179.90,193480000.0,0.0029
2009-06-12,182.69,182.74,183.72,181.23,144430000.0,0.0069
2009-06-15,180.66,182.59,182.64,178.97,136810000.0,-0.0111


In [13]:
stock.columns

Index(['Close', 'Open', 'High', 'Low', 'Volume', 'Change'], dtype='object')

In [14]:
high_prices = stock['High'].values
low_prices = stock['Low'].values
mid_prices = (high_prices + low_prices) / 2

In [15]:
seq_len = 50
sequence_length = seq_len + 1

result = []
for index in range(len(mid_prices) - sequence_length):
    result.append(mid_prices[index: index + sequence_length])

In [22]:
normalized_data = []
for window in result:
    normalized_window = [((float(p) / float(window[0])) - 1) for p in window]
    normalized_data.append(normalized_window)

result = np.array(normalized_data)

# split train and test data
row = int(round(result.shape[0] * 0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = train[:, -1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = result[row:, -1]

x_train.shape, x_test.shape

ZeroDivisionError: float division by zero

In [18]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50, 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

NotImplementedError: Cannot convert a symbolic Tensor (lstm_1/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported